In [16]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import date
from datetime import timedelta
import re
import os
import math
import sys
import functools 
import pickle

import mmx_lib as mmx
from tqdm.notebook import tqdm

from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, cross_val_predict, TimeSeriesSplit, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics.scorer import make_scorer

pd.set_option('mode.chained_assignment', None)
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

## Загрузка данных

In [17]:
#Google-trends
gt = pd.read_csv('DATA/google-trends_Trigger-MMX_recent.csv', header=0, index_col=0)
#Погода
weather = pd.read_csv('DATA/regions_weather.csv', header=0, index_col=0)

#Геоданные
geo = pd.read_csv('DATA/geo_rus.csv')

#Юскан
youscan = pd.read_csv('DATA/youscan_Trigger-MMX.csv')
youscan = youscan[youscan['fullText'].notna()&youscan['published'].notna()]
youscan = youscan.merge(geo[['Город', 'Регион']], left_on='city', right_on='Город', how='left')
youscan_no_covid = youscan[~youscan['fullText'].str.contains(r'корона|коронавирус|каронавирус|кароновирус|ковид|covid|corona', case=False,  regex=True)]
print(len(youscan), len(youscan_no_covid))

/home/analyst/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


420584 407490


## Предобработка данных

In [18]:
#Мы хотим, чтобы все индексы были в формате даты, для упрощения объединения и построения графиков
weather.index = [datetime.strptime(date+'-0', "%Y-%W-%w") for date in weather.index]
gt.index = [datetime.strptime(date, "%Y-%m-%d") for date in gt.index]

In [19]:
media = youscan_no_covid[['published', 'Регион', 'id']]
media = media.groupby(['published', 'Регион'])['id'].count().unstack().fillna(0)
media = media[[len(x)==25 for x in media.index]]
media.index = [datetime.strptime(date.split('T')[0], "%Y-%m-%d") for date in media.index]
#Уберем возникшие дупликаты дат
media = media.groupby(level=0).sum()

#Сгруппировать до недель?
media_week = media.copy()
media_week['week'] = media_week.index.map(lambda x: datetime.strftime(x, '%Y-%W'))
media_week = media_week.groupby('week').sum()
media_week.index = [datetime.strptime(date+'-0', "%Y-%W-%w") for date in media_week.index]
media_week = media_week.groupby(level=0).sum()

In [20]:
#Для моделирования отрежем 2020 год
this_year = datetime.fromisoformat('2020-01-01')
weather = weather[weather.index<this_year]
gt = gt[gt.index<this_year]
media = media[media.index<this_year]

#А это было для сравнения с остальными данными, 
#которых дальше, чем до 2017 просто нет
first_year = datetime.fromisoformat('2017-01-01')

## Наиболее коррелириующие погодные метрики

In [21]:
#Выявление зависимостей между погодными метриками и youscan
weather_long_corr = {}
for reg in tqdm(weather['Регион'].unique()):
    w = weather[weather['Регион']==reg].drop(columns=['Регион'])
    if reg in list(media_week.columns):
        weather_long_corr[reg] = {}
        for metric in w.columns:
            s1 = w[metric].groupby(level=0).mean()
            s2 = media_week[reg].groupby(level=0).sum()
            weather_long_corr[reg][metric] = mmx.long_term_correlation(s1, s2, 2, 2, 10)
        weather_long_corr[reg] = pd.Series(weather_long_corr[reg])
    elif reg in list(gt.columns):
        weather_long_corr[reg] = {}
        for metric in w.columns:
            s1 = w[metric].groupby(level=0).mean()
            s2 = gt[reg].groupby(level=0).sum()
            weather_long_corr[reg][metric] = mmx.long_term_correlation(s1, s2, 2, 2, 10)
        weather_long_corr[reg] = pd.Series(weather_long_corr[reg])
        
weather_long_corr = pd.DataFrame(weather_long_corr)

In [22]:
#Поиск самой похожей погодной метрики
metric_dict = {}
for reg in weather['Регион'].unique():
    if (reg in list(media.columns))|(reg in list(gt.columns)):
        metric_max = weather_long_corr[reg].idxmax()
        if weather_long_corr.loc[metric_max, reg]>0.1:
            metric_dict[reg] = metric_max

Чем двухнедельный прогноз отличается от недельного? Да в общем-то особо ничем. Только тем, на сколько строится предсказание.

## Значимость признаков

In [8]:
g_list = ['Камчатский край', 'Сахалинская область']

In [9]:
def maximum_search(df, alpha, weeks, period=1):
    #сравним текущее значение со средними
    nearby_median_long = df.rolling(360, min_periods=30).median()
    nearby_median_short = df.rolling(90, min_periods=7).median()
    df_delta = ((df - nearby_median_long)/nearby_median_long>alpha)|((df - nearby_median_short)/nearby_median_short>alpha)

    #мы хотим отслеживать моменты, когда значения не слишком высоки, но тем не менее стабильно растут
    df_diff = df.diff()
    df_growing = ((df_diff>0).rolling(7*period).sum()==7*period)&((df - nearby_median_long)/nearby_median_long>0)
    df_delta = df_delta|df_growing

    #мы готовы допускать пропуски не более, чем в три дня (чем определено?)
    df_holes = mmx.find_range_of_holes(df_delta, (1,4))
    df_delta = df_delta|df_holes

    #мы не хотим пики длиной меньше недели
    df_short = mmx.find_range_of_holes(~df_delta, (1, (7*period)+1))
    df_delta = df_delta&(~df_short)

    #подсчет производных                
    df_diff = df_diff.rolling(window=7*period).mean()                
    df_diff = mmx.series_norm(df_diff)
    df_diff2 = df_diff.diff().rolling(window=7*period).mean()
    df_diff2 = mmx.series_norm(df_diff2)

    df_week = df.rolling(window=7*period).mean()/nearby_median_short

    #Почему так расставлена перменная weeks?
    #Потому что для двухнедельного прогноза мы хотим знать, 
    #что к концу второй недели начнется подъем активности, 
    #нас интересует именно конец этого диапазона

    df_target = (df_delta.shift(-7*period*weeks).rolling(window=7*period).sum()>3*period).astype(int)
    return (df_target, df_delta, df_diff, df_diff2, df_week)

def region_tuning_weights(weeks=1, num=5):

    region_scores = {}
    predictions = {}
    main_feature = {}
    
    for reg in tqdm(sorted(list(geo['Регион'].unique()))):
#    for reg in tqdm(['Сахалинская область']):
        region_scores[reg] = np.nan
        predictions[reg] = np.nan
        main_feature[reg] = np.nan
        media_fullness = np.nan
        
        flag = True
        m_flag = True
       
        if (reg in list(media.columns))&(reg not in g_list):
            
        
            m = media[reg]
            #Проверим средний объем данных (не учитывая этот год, так как он, во-первых, все поломал,
            #а, во-вторых, модели обучались на данных без учета 2020 года)
            this_year = datetime.fromisoformat('2020-01-01')
            m_check = m[m.index<this_year]
            m_check = m_check.sum()/len(m_check)
            #0.1 - экспериментально полученная величина, при которой еще можно построить предсказание
            if m_check>0.1:
                #Проверка пройдень, модель строится на медийных данных, флаг переключился
                m_flag = False
                #Подгрузим остальные данные, если они есть
                if reg in list(gt.columns):
                    g = gt[reg]
                else:
                    g = pd.Series()

                if (reg in list(weather['Регион']))&(reg in list(metric_dict.keys())):
                    w = weather[weather['Регион']==reg][metric_dict[reg]]
                    w = w.groupby(level=0).mean()
                else:
                    w = pd.Series()

                #Мы не хотим учитывать слишком большие всплески.
                std = m.std()
                mean = m.mean()
                m = m.apply(lambda x: min(x, mean+3*std))

                #Несмотря на то, что мы берем разбивку по дням, лучше данные усреднить
                m = m.rolling(window=14).mean()
                #А также все отнормировать
                m = mmx.series_norm(m)
                w = mmx.series_norm(w)
                g = mmx.series_norm(g)
                #На всякий случай, при выгрузке иногда случаются дупликаты дат
                g = g.groupby(level=0).mean()                                    

                alpha = 0.3
                #Для соцмедиа данных и для google trends посчитаем всякие полезные признаки
                m_target, m_delta, m_diff, m_diff2, m_week = maximum_search(m, alpha, weeks, period=1)
                g_target, g_delta, g_diff, g_diff2, g_week = maximum_search(g, alpha, weeks, period=2)
                #А если данных в регионе маловато, добавим к медийной целевой перменной gt
                #if (m_check<1)&(reg in list(gt.columns)): 
                if (m_check<1):                    
                    m_target = (m_target+g_target)>0
                    m = m+g
                    m = mmx.series_norm(m)
                #Мы хотим собрать такой датасет только из того, что действительно есть                   
                if (reg in list(weather['Регион']))&(reg in list(metric_dict.keys()))&(reg in list(gt.columns)):
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, \
                                   w.shift(1), \
                                   g_week, g_diff<0, g_diff>0, g_diff2, g_delta], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                     'weather', \
                                     'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2', 'gdelta']
                elif reg in list(gt.columns):
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, \
                                   g_week, g_diff<0, g_diff>0, g_diff2, g_delta], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                    'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2', 'gdelta']
                elif (reg in list(weather['Регион']))&(reg in list(metric_dict.keys())):
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, w.shift(1)], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                     'weather']
                else:
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta']

        #А вот что, если соцмедиа данных не хватает?
        if m_flag:
            #Пробуем строить на основе погоды и gt
            if ((reg in list(gt.columns))&((reg in list(weather['Регион']))&(reg in list(metric_dict.keys())))&m_flag)|(reg in g_list):
                w = weather[weather['Регион']==reg][metric_dict[reg]]
                w = w.groupby(level=0).mean()            
                w = mmx.series_norm(w)
                g = gt[reg]
                g = mmx.series_norm(g)

                alpha = 0.3
                g_target, g_delta, g_diff, g_diff2, g_week = maximum_search(g, alpha, weeks, period=1)
                x = pd.concat([w, w.rolling(4).mean(), w.diff(), g_delta, g_week, g_diff<0, g_diff>0, g_diff2], axis=1)
                feature_names = ['weather', 'wmean', 'wdiff', 'gdelta', 'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2']
                #тогда целевой тпеременно будет только Gt
                m_target = g_target.copy()
                #и притворимся, что gt - это медиаданные 
                #(это нужно для корректного выведения данных в дэшборде)
                m = g.copy()
            else:
                flag=False

        if flag:
            x.columns = feature_names
            x = x.replace([np.inf, -np.inf], np.nan)
            x = x.fillna(method='ffill')
            x = x.astype(float).dropna()
            x = (x[x.index.isin(m_target.index)]).astype(float).dropna()
            y = m_target.copy()
            y = y[y.index.isin(x.index)]

            #Для каждого региона модель нужно настраивать отдельно, есть такое дело.
            estimators = {
                'rf': (RandomForestClassifier(), 
                       {'n_estimators': [100, 120, 140, 160, 180, 200], 'max_depth': [5, 10, 15, 20]}),
                'svc': (SVC(C=1), 
                        {'gamma': ['scale', 'auto', 2]}),
                'gb': ( GradientBoostingClassifier(), 
                       {'n_estimators': [500, 1000, 2000, 3000, 5000]})                    
            }
            
            try:

                params = {}
                y_pred_gs = []
                for est in estimators:
                    clf = estimators[est][0]
                    params_dict = estimators[est][1]
                    gs = GridSearchCV(clf, params_dict, scoring=mmx.delay_score)
                    gs.fit(x, y)
                    y_pred_gs.append(gs.predict(x))
                    params[est] = gs.best_params_

                final_params = {
                    'n_estimators': [100, 120, 140, 160, 180, 200],
                    'max_depth': [5, 10, 15, 20]
                }                
                clf = RandomForestRegressor()
                gs = GridSearchCV(clf, final_params, scoring=mmx.delay_score)
                gs.fit(pd.DataFrame(y_pred_gs).T, y)
                final_params = gs.best_params_

                n_splits = 10
                kf = KFold(n_splits=n_splits, random_state=None, shuffle=False)

                #Построение предсказаний
                y_pred_total = []
                feat_seed = [0]*len(x.columns)
                feat_weights = dict(zip(list(estimators.keys()), [feat_seed]*len(estimators)))
                final_regr_weight = [0]*len(estimators)
                est_exception = dict(zip(list(estimators.keys()), [False]*len(estimators)))
                #Кросс-валидация на нересекающихся неперемшанных промежутках
                #Почему это важно и почему можно так?
                #Потому что наш прогноз основан на данных за последнюю пару недель и нехорошо их совсем перемешивать
                #С другой стороны, сама целевая переменная не используется в качестве признаков,
                #Так что можно не опасаться использовать в обучении данные из будущего - они никак не помешают настоящему
                for train_index, test_index in kf.split(x):
                    X_train, X_test = x.iloc[train_index, :], x.iloc[test_index, :]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                    #заготовим перменные под промеуточные результаты
                    y_pred_i = []
                    y_train_i = []
                    for est in estimators:
                        clf = estimators[est][0].set_params(**params[est])
                        clf.fit(X_train, y_train)
                        #Так же мы хотим оценивать веса признаков в моделях для разных регионов
                        try:
                            feat_weights[est] = [x+y for x, y in zip(feat_weights[est], clf.feature_importances_)]
                        except:
                            est_exception[est] = True
                        y_train_i.append(clf.predict(X_train))
                        y_pred_i.append(clf.predict(X_test))                    

                    regr_total = RandomForestRegressor().set_params(**final_params)                          
                    regr_total.fit(pd.DataFrame(y_train_i).T, 
                                   y_train)
                    final_regr_weight = [x+y for x, y in zip(final_regr_weight, regr_total.feature_importances_)]
                    y_pred_total.append(pd.Series(
                        regr_total.predict(pd.DataFrame(y_pred_i).T), 
                        index=X_test.index))

                feat_weights = {est: [val/n_splits for val in val_list] for est, val_list in feat_weights.items()}
                final_regr_weight = [val/n_splits for val in final_regr_weight]
                print(final_regr_weight)
                for i, est in enumerate(feat_weights.keys()):
                    feat_weights[est] = [w*final_regr_weight[i] for w in feat_weights[est]]
                feat_weights_final = [sum(i) for i in zip(*list(feat_weights.values()))]
                y_pred = pd.concat(y_pred_total).sort_index() 

                y_int = (y_pred>0.5).astype(int)
                y_int = y_int[y_int.index.isin(m_target.index)]
                m_target = m_target[m_target.index.isin(y_int.index)]

                region_scores[reg] = mmx.delay_func(y_int, m_target)
                predictions[reg] = pd.concat([y_pred, y], axis=1)
                predictions[reg].columns = ['y_pred', 'y']
                main_feature[reg] = dict(zip(feature_names, feat_weights_final))
            except:
                print(reg, sys.exc_info()[:2])

    for est in est_exception:
        if est_exception[est]:
            print(f'Для {est} нет возможности вычислить значимость признаков')
            
        
    return region_scores, predictions, main_feature, media_fullness

In [10]:
#region_scores, predictions, main_feature, media_fullness = region_tuning_weights(weeks=1)
region_scores_2, predictions_2, main_feature_2, media_fullness_2 = region_tuning_weights(weeks=2)

KeyboardInterrupt: 

Построить картиночки

## Обучение и сохранение моделей

In [23]:
with open('metric_dict.pickle', 'wb') as f:
                        # Pickle the 'data' dictionary using the highest protocol available.
                        pickle.dump(metric_dict, f, pickle.HIGHEST_PROTOCOL)

In [24]:
g_list = ['Камчатский край', 'Сахалинская область']

In [25]:
def maximum_search(df, alpha, weeks, period=1):
    #сравним текущее значение со средними
    nearby_median_long = df.rolling(360, min_periods=30).median()
    nearby_median_short = df.rolling(90, min_periods=7).median()
    df_delta = ((df - nearby_median_long)/nearby_median_long>alpha)|((df - nearby_median_short)/nearby_median_short>alpha)

    #мы хотим отслеживать моменты, когда значения не слишком высоки, но тем не менее стабильно растут
    df_diff = df.diff()
    df_growing = ((df_diff>0).rolling(7*period).sum()==7*period)&((df - nearby_median_long)/nearby_median_long>0)
    df_delta = df_delta|df_growing

    #мы готовы допускать пропуски не более, чем в три дня (чем определено?)
    df_holes = mmx.find_range_of_holes(df_delta, (1,4))
    df_delta = df_delta|df_holes

    #мы не хотим пики длиной меньше недели
    df_short = mmx.find_range_of_holes(~df_delta, (1, (7*period)+1))
    df_delta = df_delta&(~df_short)

    #подсчет производных                
    df_diff = df_diff.rolling(window=7*period).mean()                
    df_diff = mmx.series_norm(df_diff)
    df_diff2 = df_diff.diff().rolling(window=7*period).mean()
    df_diff2 = mmx.series_norm(df_diff2)

    df_week = df.rolling(window=7*period).mean()/nearby_median_short

    #Почему так расставлена перменная weeks?
    #Потому что для двухнедельного прогноза мы хотим знать, 
    #что к концу второй недели начнется подъем активности, 
    #нас интересует именно конец этого диапазона

    df_target = (df_delta.shift(-7*period*weeks).rolling(window=7*period).sum()>3*period).astype(int)
    return (df_target, df_delta, df_diff, df_diff2, df_week)
    
    
def region_save_models(weeks=1, num=5):
    region_scores = {}
    predictions = {}
    main_feature = {}
    
    for reg in tqdm(sorted(list(geo['Регион'].unique()))):
#     for reg in tqdm(['Московская область', 'Пензенская область', 'Тверская область', 'Хабаровский край']):
        region_scores[reg] = np.nan
        predictions[reg] = np.nan
        main_feature[reg] = np.nan
        media_fullness = np.nan
        
        flag = True
        m_flag = True
       
        if (reg in list(media.columns))&(reg not in g_list):
            
        
            m = media[reg]
            #Проверим средний объем данных (не учитывая этот год, так как он, во-первых, все поломал,
            #а, во-вторых, модели обучались на данных без учета 2020 года)
            this_year = datetime.fromisoformat('2020-01-01')
            m_check = m[m.index<this_year]
            m_check = m_check.sum()/len(m_check)
            #0.1 - экспериментально полученная величина, при которой еще можно построить предсказание
            if m_check>0.1:
                #Проверка пройдень, модель строится на медийных данных, флаг переключился
                m_flag = False
                #Подгрузим остальные данные, если они есть
                if reg in list(gt.columns):
                    g = gt[reg]
                else:
                    g = pd.Series()

                if (reg in list(weather['Регион']))&(reg in list(metric_dict.keys())):
                    w = weather[weather['Регион']==reg][metric_dict[reg]]
                    w = w.groupby(level=0).mean()
                else:
                    w = pd.Series()

                #Мы не хотим учитывать слишком большие всплески.
                std = m.std()
                mean = m.mean()
                m = m.apply(lambda x: min(x, mean+3*std))

                #Несмотря на то, что мы берем разбивку по дням, лучше данные усреднить
                m = m.rolling(window=14).mean()
                #А также все отнормировать
                m = mmx.series_norm(m)
                w = mmx.series_norm(w)
                g = mmx.series_norm(g)
                #На всякий случай, при выгрузке иногда случаются дупликаты дат
                g = g.groupby(level=0).mean()                                    

                alpha = 0.3
                #Для соцмедиа данных и для google trends посчитаем всякие полезные признаки
                m_target, m_delta, m_diff, m_diff2, m_week = maximum_search(m, alpha, weeks, period=1)
                g_target, g_delta, g_diff, g_diff2, g_week = maximum_search(g, alpha, weeks, period=2)
                #А если данных в регионе маловато, добавим к медийной целевой перменной gt
                #if (m_check<1)&(reg in list(gt.columns)): 
                if (m_check<1):                    
                    m_target = (m_target+g_target)>0
                    m = m+g
                    m = mmx.series_norm(m)
                #Мы хотим собрать такой датасет только из того, что действительно есть                   
                if (reg in list(weather['Регион']))&(reg in list(metric_dict.keys()))&(reg in list(gt.columns)):
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, \
                                   w.shift(1), \
                                   g_week, g_diff<0, g_diff>0, g_diff2, g_delta], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                     'weather', \
                                     'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2', 'gdelta']
                elif reg in list(gt.columns):
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, \
                                   g_week, g_diff<0, g_diff>0, g_diff2, g_delta], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                    'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2', 'gdelta']
                elif (reg in list(weather['Регион']))&(reg in list(metric_dict.keys())):
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, w.shift(1)], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                     'weather']
                else:
                    x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta], axis=1)
                    feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta']

        #А вот что, если соцмедиа данных не хватает?
        if m_flag:
            #Пробуем строить на основе погоды и gt
            if ((reg in list(gt.columns))&((reg in list(weather['Регион']))&(reg in list(metric_dict.keys())))&m_flag)|(reg in g_list):
                w = weather[weather['Регион']==reg][metric_dict[reg]]
                w = w.groupby(level=0).mean()            
                w = mmx.series_norm(w)
                g = gt[reg]
                g = mmx.series_norm(g)

                alpha = 0.3
                g_target, g_delta, g_diff, g_diff2, g_week = maximum_search(g, alpha, weeks, period=1)
                x = pd.concat([w, w.rolling(4).mean(), w.diff(), g_delta, g_week, g_diff<0, g_diff>0, g_diff2], axis=1)
                feature_names = ['weather', 'wmean', 'wdiff', 'gdelta', 'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2']
                #тогда целевой тпеременно будет только Gt
                m_target = g_target.copy()
                #и притворимся, что gt - это медиаданные 
                #(это нужно для корректного выведения данных в дэшборде)
                m = g.copy()
            else:
                flag=False

        #Так, если у нас ничего не сломалось, то строим предсказание
        if flag:  
            x.columns = feature_names
            x = x.replace([np.inf, -np.inf], np.nan)
            x = x.fillna(method='ffill')
            x = x.astype(float).dropna()
            #Севастополь - особенный город... 
            #у него до 16 октября 2018 года почему-то беда с данными
            #на всякий случай уберем их руками
            if reg=='Севастополь':
                last_year = datetime.fromisoformat('2018-10-16')
                x = x[x.index>last_year]

            m_target = m_target.groupby(level=0).mean()
            x = x.groupby(level=0).mean()
            y = m_target.copy()                
            y = y[y.index.isin(x.index)]
            x = x[x.index.isin(y.index)]

            #Для каждого региона модель нужно настраивать отдельно, есть такое дело.
            estimators = {
                'rf': [RandomForestClassifier(), 
                       {'n_estimators': [100, 120, 140, 160, 180, 200], 'max_depth': [5, 10, 15, 20]}],
                'svc': [SVC(C=1), 
                        {'gamma': ['scale', 'auto', 2]}],
                'gb': [GradientBoostingClassifier(), 
                       {'n_estimators': [500, 1000, 2000, 3000, 5000]}]                    
            }
            #Подблор параметров для ансамбля моделей
            try:
                params = {}
                y_pred_gs = []
                for est in estimators:
                    clf = estimators[est][0]
                    params_dict = estimators[est][1]
                    gs = GridSearchCV(clf, params_dict, scoring=mmx.delay_score)
                    gs.fit(x, y)
                    y_pred_gs.append(gs.predict(x))
                    params[est] = gs.best_params_
                #подбор параметров для итогового регрессора
                final_params = {
                    'n_estimators': [100, 120, 140, 160, 180, 200],
                    'max_depth': [5, 10, 15, 20]
                }                
                clf = RandomForestRegressor()
                gs = GridSearchCV(clf, final_params, scoring=mmx.delay_score)
                gs.fit(pd.DataFrame(y_pred_gs).T, y)
                final_params = gs.best_params_
                for est in estimators:
                        estimators[est][0] = estimators[est][0].set_params(**params[est])
                regr_total = RandomForestRegressor().set_params(**final_params)

                #Обучение выбранных моделей
                y_est = []
                est_names = sorted(list(estimators.keys()))
                for est in est_names:
                    clf = estimators[est][0]
                    clf.fit(x, y)
                    estimators[est][0] = clf
                    y_est.append(clf.predict(x))
                    #сохраним модели в pickle
                    with open(f'models/testing/{reg}_{est}_{weeks}week.pickle', 'wb') as f:
                        # Pickle the 'data' dictionary using the highest protocol available.
                        pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

                regr_total.fit(pd.DataFrame(y_est).T, y)
                with open(f'models/testing/{reg}_finalregr_{weeks}week.pickle', 'wb') as f:
                    # Pickle the 'data' dictionary using the highest protocol available.
                    pickle.dump(regr_total, f, pickle.HIGHEST_PROTOCOL)
                    
                with open(f'models/testing/{reg}_colnames_{weeks}week.pickle', 'wb') as f:
                    # Pickle the 'data' dictionary using the highest protocol available.
                    pickle.dump(feature_names, f, pickle.HIGHEST_PROTOCOL)
            except:
                print(reg, sys.exc_info()[:2])



In [26]:
region_save_models(weeks=1)

Якутия (<class 'ValueError'>, ValueError('The number of classes has to be greater than one; got 1 class'))



In [ ]:
region_save_models(weeks=2)

Якутия (<class 'ValueError'>, ValueError('The number of classes has to be greater than one; got 1 class'))
